IMPORTATIONS

In [3]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import euclidean

PARAMÈTRES GÉNÉRAUX

In [9]:
data_path = Path("C:/Users/bfontaine/Downloads/données SAE 4.02")
equip_path = data_path / "Ensemble-com-2021_csv"

IMPORTATIONS

In [66]:
df_communes = pd.read_excel(data_path / "grille_densite_7_niveaux_2024.xlsx")
def charger_equipements(nom_fichier):
    df = pd.read_csv(equip_path / nom_fichier, sep=";", dtype=str)
    
    possible_cols = ["CODGEO", "CODG_COM", "Code_INSEE", "COMMUNE", "INSEE"]
    code_col = next((col for col in possible_cols if col in df.columns), None)
    
    if code_col is None:
        raise ValueError(f"Aucune colonne de code INSEE trouvée dans {nom_fichier}")
    
    return df.groupby(code_col).size().rename(nom_fichier)


TRAITEMENTS COMMUNES

In [72]:
#filtrer les communes entre 10 et 20k habitants
df_communes = df_communes[(df_communes["PMUN21"] >= 10000) & (df_communes["PMUN21"] <= 20000)]

#on séléctionne les colonnes qu'on veut
df_base = df_communes[["CODGEO", "LIBGEO", "PMUN21", "DENS"]].copy()
df_base.columns = ["CODGEO", "LIBGEO", "PMUN21", "DENS"]


DICTIONNAIRE CONTENANT LES FICHIERS PAR DOMAINE

In [73]:
fichiers_equipements = {
    "Nb_Equip_Santé": [
        "equip-serv-sante-com-2021.csv",
        "equip-serv-medical-para-com-2021.csv"
    ],
    "Nb_Equip_Education": [
        "equip-serv-ens-1er-degre-com-2021.csv",
        "equip-serv-ens-2eme-degre-com-2021.csv",
        "equip-serv-ens-sup-form-serv-com-2021.csv"
    ],
    "Nb_Equip_Sport_Loisir": ["equip-sport-loisir-socio-com-2021.csv"],
    "Nb_Equip_Culture": ["equip-sport-loisir-socio-com-2021.csv"],
    "Nb_Equip_Services": ["equip-serv-particuliers-com-2021.csv"],
    "Nb_Equip_Commerce": ["equip-serv-commerce-com-2021.csv"],
    "Nb_Equip_Transport": ["equip-tour-transp-com-2021.csv"],
    "Nb_Equip_Social": ["equip-serv-action-sociale-com-2021.csv"]
}


CALCULER LES EQUIPEMENTS PAR DOMAINE

In [ ]:
#mise en place index
df_base = df_base.set_index("CODGEO")
df_equipements = pd.DataFrame(index=df_base.index)

#boucle pour chaque domaine
for domaine, fichiers in fichiers_equipements.items():
    equip_series = [charger_equipements(fichier) for fichier in fichiers]
    total = pd.concat(equip_series, axis=1).sum(axis=1)
    df_equipements[domaine] = total

#remplir les colonnes vide par 0
df_equipements = df_equipements.fillna(0)

FUSION AVEC LA BASE PRINCIPALE

In [75]:
df_base = df_base.merge(df_equipements, left_index=True, right_index=True, how="left")
df_base = df_base.fillna(0)

CALCUL DES RATIOS PAR HABITANTS

In [80]:
print(df_base.columns)
domaines = [col for col in df_base.columns if col.startswith("Nb_Equip_")]
for domaine in domaines:
    df_base[f"{domaine}_par_1000h"] = df_base[domaine] / df_base["PMUN21"] * 1000

Index(['LIBGEO', 'PMUN21', 'DENS', 'Nb_Equip_Santé_x', 'Nb_Equip_Education_x',
       'Nb_Equip_Sport_Loisir_x', 'Nb_Equip_Culture_x', 'Nb_Equip_Services_x',
       'Nb_Equip_Commerce_x', 'Nb_Equip_Transport_x', 'Nb_Equip_Social_x',
       'Nb_Equip_Santé_y', 'Nb_Equip_Education_y', 'Nb_Equip_Sport_Loisir_y',
       'Nb_Equip_Culture_y', 'Nb_Equip_Services_y', 'Nb_Equip_Commerce_y',
       'Nb_Equip_Transport_y', 'Nb_Equip_Social_y',
       'Nb_Equip_Santé_x_par_1000h', 'Nb_Equip_Education_x_par_1000h',
       'Nb_Equip_Sport_Loisir_x_par_1000h', 'Nb_Equip_Culture_x_par_1000h',
       'Nb_Equip_Services_x_par_1000h', 'Nb_Equip_Commerce_x_par_1000h',
       'Nb_Equip_Transport_x_par_1000h', 'Nb_Equip_Social_x_par_1000h',
       'Nb_Equip_Santé_y_par_1000h', 'Nb_Equip_Education_y_par_1000h',
       'Nb_Equip_Sport_Loisir_y_par_1000h', 'Nb_Equip_Culture_y_par_1000h',
       'Nb_Equip_Services_y_par_1000h', 'Nb_Equip_Commerce_y_par_1000h',
       'Nb_Equip_Transport_y_par_1000h', 'Nb_Equip

INDICATEURS SYNTHESES ET DESEQUILIBRES

In [82]:
df_base["Equip_Totaux"] = df_base[domaines].sum(axis=1)
df_base["Equip_Total_par_1000h"] = df_base["Equip_Totaux"] / df_base["PMUN21"] * 1000
df_base["Ratio_Education_Santé"] = df_base["Nb_Equip_Education_x"] / df_base["Nb_Equip_Santé_y"].replace(0, np.nan)
df_base["Ratio_Sport_Culture"] = df_base["Nb_Equip_Sport_Loisir_y"] / df_base["Nb_Equip_Culture_y"].replace(0, np.nan)
df_base["Ecart_Max_Min"] = df_base[domaines].max(axis=1) - df_base[domaines].min(axis=1)
df_base["Diversité_Equipements"] = (df_base[domaines] > 0).sum(axis=1)

EXPORT

In [84]:
df_base.to_csv("C:/Users/bfontaine/Downloads/données SAE 4.02/base_communes_SAE402.csv", index=False, encoding="utf-8-sig")